In [2]:
import spacy
from spacy.training.example import Example #Müssen später vermutlich was anderes nehmen
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import os

# Load the base model
nlp = spacy.blank("en")
nlp = spacy.blank("en")

# Create and add the NER pipeline component
ner = nlp.add_pipe("ner")

# Define the labels and add them to the NER component
labels = ["PERSON"]
for label in labels:
    ner.add_label(label)

# Define the training data
train_data = [
    (
        "(Queen Sansa Star is the eldest daughter of Lord Eddard Stark and his wife, Lady Catelyn, sister of Robb, Arya, Bran, and Rickon Stark, and half-sister of Jon Snow; though truthfully is his cousin,",
        {"entities": [(6, 16, "PERSON"), (48, 60, "PERSON"), (75, 87, "PERSON"), (120, 133, "PERSON"), (154, 162, "PERSON")]},
    )
]

# Check entity alignment
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    biluo_tags = offsets_to_biluo_tags(doc, annotations["entities"])

# Initialize the optimizer
optimizer = nlp.initialize()

# Perform the training
for iteration in range(10):  #10 für die kleine Anzahl. Ansonsten könnte man ja so lange iterieren, bis die Fehler eine gewisse Untergrenze erreicht haben
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, sgd=optimizer)
    print("Losses:", losses)

# Save the trained NER model
output_dir = os.getcwd()
nlp.to_disk(output_dir)

# Load the saved model
loaded_nlp = spacy.load(output_dir)

# Test the loaded NER model
test_text = "(Queen Sansa Star is the eldest daughter of Lord Eddard Stark and his wife, Lady Catelyn, sister of Robb, Arya, Bran, and Rickon Stark, and half-sister of Jon Snow; though truthfully is his cousin,"
doc = loaded_nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent.label_) #Laut Recherche leer, weil Datensatz zu klein


c:\Users\Adriana\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "(Queen Sansa Star is the eldest daughter of Lord E..." with entities "[(6, 16, 'PERSON'), (48, 60, 'PERSON'), (75, 87, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 30.166668072342873}
Losses: {'ner': 28.754965037107468}
Losses: {'ner': 26.269851714372635}
Losses: {'ner': 22.492678180336952}
Losses: {'ner': 16.867720879614353}
Losses: {'ner': 10.381545104086399}
Losses: {'ner': 4.357643008232117}
Losses: {'ner': 1.1866512128617615}
Losses: {'ner': 0.16519744190736674}
Losses: {'ner': 0.008519274231730378}


In [3]:
import json

# Laden der validierten Daten aus dem JSON-Dateiformat
with open('relationships.json', 'r') as file:
    validation_data = json.load(file)

# Ausgabe des "data" Objekts
print(validation_data)

# Iteriere über die Sätze im Validierungsdatensatz

for data in validation_data:
    sentence = data['sentence']
    entities = data['relationships']
    predicted_relationships=[]
    for entity in entities:
        firstEntity = entity['firstEntity']
        #relationshipType = entity['relationshipType'] # bei ner ja eignetlich unnötig
        secondEntity = entity['firstEntity']
        predicted_relationships.append(firstEntity)
        predicted_relationships.append(secondEntity)


    # Verarbeite den Satz mit dem geladenen NER-Modell
    doc = loaded_nlp(sentence)

    # Vergleiche die vorhergesagten Entitäten mit den tatsächlichen Entitäten
    predicted_entities = [(ent.text, ent.label_) for ent in doc.ents] # wird leer sein (war oben schon leer) , benötigen einen größeren Datensatz
    if predicted_entities == entities: # geht vermutlich bei arrays nicht
        print("Entitäten: Vorhersage stimmt überein")
    else:
        print("Entitäten: Vorhersage stimmt nicht überein")



[{'sentence': 'Queen Sansa Stark is the eldest daughter of Lord Eddard Stark and his wife, Lady Catelyn, sister of Robb, Arya, Bran, and Rickon Stark, and "half-sister" of Jon Snow; though truthfully is his cousin', 'relationships': [{'firstEntity': 'Sansa Stark', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntity': 'Sansa Stark', 'relationshipType': 'Siblings', 'secondEntity': 'Jon Snow'}, {'firstEntity': 'Sansa Stark', 'relationshipType': 'Siblings', 'secondEntity': 'Rickon Stark'}, {'firstEntity': 'Jon Snow', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Jon Snow'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Sansa Stark'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Rickon Stark'}, {'firstEntity': 'Rickon Stark', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntit